# Aula 4: Implantação de Modelos (Deployment)

## Objetivos de Aprendizagem
- Registrar modelos no MLFlow Model Registry
- Versionar modelos adequadamente
- Servir modelos via API REST
- Implementar estratégias de deployment
- Criar endpoints de inferência

## Exercício Prático
Implantar um modelo de ML em produção usando MLFlow.

## 1. Preparação do Ambiente

In [ ]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import joblib
import json
import requests
import warnings
warnings.filterwarnings('ignore')

## 2. Treinamento do Modelo para Deployment

### Tarefa 1: Treine um modelo pronto para produção

In [ ]:
# Carregar dados
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

# Dividir dados
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Dados de treino: {X_train.shape}")
print(f"Dados de teste: {X_test.shape}")
print(f"Classes: {iris.target_names}")

In [ ]:
# Configurar experimento
mlflow.set_experiment("iris_deployment")

# Treinar modelo com assinatura
with mlflow.start_run(run_name="production_model_v1") as run:
    # Pipeline de ML
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
    ])
    
    # Treinar
    pipeline.fit(X_train, y_train)
    
    # Avaliar
    train_score = pipeline.score(X_train, y_train)
    test_score = pipeline.score(X_test, y_test)
    
    # Inferir assinatura do modelo
    signature = infer_signature(X_train, pipeline.predict(X_train))
    
    # Criar exemplo de input
    input_example = X_train.iloc[:5]
    
    # Registrar parâmetros e métricas
    mlflow.log_params({
        "model_type": "RandomForest",
        "n_estimators": 100,
        "n_features": X_train.shape[1]
    })
    mlflow.log_metrics({
        "train_accuracy": train_score,
        "test_accuracy": test_score
    })
    
    # Registrar modelo com assinatura e exemplo
    mlflow.sklearn.log_model(
        pipeline,
        "model",
        signature=signature,
        input_example=input_example,
        registered_model_name="iris_classifier"
    )
    
    run_id = run.info.run_id
    
    print(f"\nModelo treinado!")
    print(f"Run ID: {run_id}")
    print(f"Train Accuracy: {train_score:.4f}")
    print(f"Test Accuracy: {test_score:.4f}")

## 3. Gerenciamento de Modelos no Registry

### Tarefa 2: Promova o modelo para diferentes estágios

In [ ]:
# Obter cliente do MLFlow
client = mlflow.tracking.MlflowClient()

# Listar versões do modelo
model_name = "iris_classifier"
versions = client.search_model_versions(f"name='{model_name}'")

print(f"Versões do modelo '{model_name}':")
for version in versions:
    print(f"  Versão {version.version}: Stage = {version.current_stage}")

In [ ]:
# Promover modelo para Staging
latest_version = versions[0].version

client.transition_model_version_stage(
    name=model_name,
    version=latest_version,
    stage="Staging",
    archive_existing_versions=False
)

print(f"Modelo versão {latest_version} promovido para Staging")

In [ ]:
# Adicionar descrição e tags ao modelo
client.update_model_version(
    name=model_name,
    version=latest_version,
    description="Modelo de classificação de Iris - Versão inicial para staging"
)

client.set_model_version_tag(
    name=model_name,
    version=latest_version,
    key="validation_status",
    value="passed"
)

print("Metadados atualizados")

## 4. Carregamento de Modelos para Inferência

### Tarefa 3: Carregue e teste o modelo

In [ ]:
# Carregar modelo do registry (stage Staging)
model_uri = f"models:/{model_name}/Staging"
loaded_model = mlflow.sklearn.load_model(model_uri)

print(f"Modelo carregado de: {model_uri}")
print(f"Tipo: {type(loaded_model)}")

In [ ]:
# Fazer previsões
sample_data = X_test.iloc[:5]
predictions = loaded_model.predict(sample_data)
probabilities = loaded_model.predict_proba(sample_data)

print("\n=== PREVISÕES DE EXEMPLO ===")
for i, (pred, prob) in enumerate(zip(predictions, probabilities)):
    print(f"\nAmostra {i+1}:")
    print(f"  Classe predita: {iris.target_names[pred]}")
    print(f"  Probabilidades: {dict(zip(iris.target_names, prob))}")

## 5. Salvando Modelo Localmente

### Tarefa 4: Exporte o modelo para uso local

In [ ]:
import os

# Criar diretório para modelos
model_dir = "/tmp/deployed_models"
os.makedirs(model_dir, exist_ok=True)

# Salvar modelo
model_path = os.path.join(model_dir, "iris_model.pkl")
joblib.dump(loaded_model, model_path)

print(f"Modelo salvo em: {model_path}")
print(f"Tamanho do arquivo: {os.path.getsize(model_path) / 1024:.2f} KB")

In [ ]:
# Criar metadata do modelo
metadata = {
    "model_name": model_name,
    "version": str(latest_version),
    "features": list(X.columns),
    "target_names": list(iris.target_names),
    "training_date": pd.Timestamp.now().isoformat(),
    "metrics": {
        "test_accuracy": float(test_score)
    }
}

metadata_path = os.path.join(model_dir, "model_metadata.json")
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"\nMetadata salvo em: {metadata_path}")
print(json.dumps(metadata, indent=2))

## 6. Simulação de API de Inferência

### Tarefa 5: Crie uma função de inferência simulando uma API

In [ ]:
class ModelInferenceService:
    """Serviço de inferência simulando uma API"""
    
    def __init__(self, model_path, metadata_path):
        self.model = joblib.load(model_path)
        with open(metadata_path, 'r') as f:
            self.metadata = json.load(f)
        print(f"Serviço inicializado com modelo versão {self.metadata['version']}")
    
    def predict(self, input_data):
        """Fazer previsão"""
        # Validar input
        if not isinstance(input_data, pd.DataFrame):
            input_data = pd.DataFrame(input_data, columns=self.metadata['features'])
        
        # Previsão
        predictions = self.model.predict(input_data)
        probabilities = self.model.predict_proba(input_data)
        
        # Formatar resposta
        results = []
        for pred, prob in zip(predictions, probabilities):
            results.append({
                "prediction": self.metadata['target_names'][pred],
                "confidence": float(prob.max()),
                "probabilities": {
                    name: float(p) 
                    for name, p in zip(self.metadata['target_names'], prob)
                }
            })
        
        return results
    
    def health_check(self):
        """Verificar saúde do serviço"""
        return {
            "status": "healthy",
            "model_name": self.metadata['model_name'],
            "model_version": self.metadata['version']
        }

# Inicializar serviço
service = ModelInferenceService(model_path, metadata_path)
print("\nServiço de inferência pronto!")

In [ ]:
# Testar health check
health = service.health_check()
print("Health Check:")
print(json.dumps(health, indent=2))

In [ ]:
# Testar inferência
test_input = [
    [5.1, 3.5, 1.4, 0.2],  # Setosa
    [6.3, 2.9, 5.6, 1.8],  # Virginica
    [5.9, 3.0, 4.2, 1.5]   # Versicolor
]

predictions = service.predict(test_input)

print("\n=== RESULTADOS DA INFERÊNCIA ===")
for i, pred in enumerate(predictions, 1):
    print(f"\nInput {i}:")
    print(json.dumps(pred, indent=2))

## 7. Versionamento e Rollback

### Tarefa 6: Simule o deployment de uma nova versão

In [ ]:
# Treinar nova versão do modelo
with mlflow.start_run(run_name="production_model_v2") as run:
    # Pipeline com parâmetros diferentes
    pipeline_v2 = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42))
    ])
    
    pipeline_v2.fit(X_train, y_train)
    test_score_v2 = pipeline_v2.score(X_test, y_test)
    
    # Registrar modelo
    mlflow.sklearn.log_model(
        pipeline_v2,
        "model",
        signature=signature,
        registered_model_name=model_name
    )
    
    print(f"Nova versão treinada!")
    print(f"Test Accuracy v2: {test_score_v2:.4f}")
    print(f"Test Accuracy v1: {test_score:.4f}")

In [ ]:
# Comparar versões
versions = client.search_model_versions(f"name='{model_name}'")

print("\n=== VERSÕES DO MODELO ===")
for v in sorted(versions, key=lambda x: int(x.version), reverse=True):
    print(f"Versão {v.version}:")
    print(f"  Stage: {v.current_stage}")
    print(f"  Status: {v.status}")
    print(f"  Run ID: {v.run_id}")
    print()

## 8. Promovendo para Produção

### Tarefa 7: Promova o melhor modelo para produção

In [ ]:
# Promover versão atual para produção
client.transition_model_version_stage(
    name=model_name,
    version=latest_version,
    stage="Production",
    archive_existing_versions=True
)

print(f"Modelo versão {latest_version} promovido para Production")
print("Versões anteriores arquivadas")

In [ ]:
# Carregar modelo de produção
production_model = mlflow.sklearn.load_model(f"models:/{model_name}/Production")

print("Modelo de produção carregado com sucesso!")
print(f"Accuracy no teste: {production_model.score(X_test, y_test):.4f}")

## 9. Estratégias de Deployment

### Conceitos Importantes:

#### Blue-Green Deployment
- Manter dois ambientes (azul e verde)
- Trocar tráfego instantaneamente
- Rollback rápido se necessário

#### Canary Deployment
- Liberar gradualmente para um subconjunto de usuários
- Monitorar performance
- Aumentar tráfego progressivamente

#### Shadow Deployment
- Executar novo modelo em paralelo
- Não afetar usuários
- Comparar resultados antes de trocar

In [ ]:
# Simulação de Canary Deployment
class CanaryDeployment:
    def __init__(self, model_current, model_canary, canary_percentage=10):
        self.model_current = model_current
        self.model_canary = model_canary
        self.canary_percentage = canary_percentage
        self.requests_count = 0
    
    def predict(self, X):
        self.requests_count += 1
        
        # Decidir qual modelo usar
        if np.random.rand() * 100 < self.canary_percentage:
            model_used = "canary"
            prediction = self.model_canary.predict(X)
        else:
            model_used = "current"
            prediction = self.model_current.predict(X)
        
        return prediction, model_used

# Simular canary deployment
canary = CanaryDeployment(loaded_model, production_model, canary_percentage=20)

print("\n=== SIMULAÇÃO DE CANARY DEPLOYMENT (20%) ===")
canary_count = 0
current_count = 0

for i in range(100):
    sample = X_test.iloc[[i % len(X_test)]]
    pred, model_used = canary.predict(sample)
    if model_used == "canary":
        canary_count += 1
    else:
        current_count += 1

print(f"Requests para modelo atual: {current_count}")
print(f"Requests para modelo canary: {canary_count}")
print(f"Porcentagem canary: {canary_count}%")

## 10. Exercícios Adicionais

### Desafios para Praticar:

1. **MLFlow Model Serving**: Inicie o MLFlow model serving localmente:
   ```bash
   mlflow models serve -m models:/iris_classifier/Production -p 5001
   ```

2. **API REST**: Crie requests para o endpoint do MLFlow:
   ```python
   import requests
   data = {"instances": [[5.1, 3.5, 1.4, 0.2]]}
   response = requests.post("http://localhost:5001/invocations", json=data)
   ```

3. **Docker**: Crie uma imagem Docker do modelo
4. **A/B Testing**: Implemente lógica de A/B testing entre modelos
5. **Batch Inference**: Crie pipeline para inferência em lote
6. **Feature Store**: Simule integração com feature store

### Questões para Reflexão:

1. Qual estratégia de deployment é mais adequada para seu caso de uso?
2. Como você garantiria zero downtime durante o deployment?
3. Quais métricas você monitoraria em produção?
4. Como você implementaria rollback automático?